In [279]:
from neo4j import GraphDatabase
import random
import pandas as pd
from dotenv import load_dotenv
import os

In [280]:
load_dotenv()

uri = os.getenv('NEO4J_URI')
user = os.getenv('NEO4J_USER')
password = os.getenv('NEO4J_PASSWORD')
database = os.getenv('NEO4J_DATABASE')

driver = GraphDatabase.driver(uri, auth=(user, password), database=database)

In [281]:
def get_colors():
    records, _, _ = driver.execute_query("""
        MATCH (n:Team)--()
        WITH n, count(*) AS degree
        WITH max(degree) AS maxDegree
        RETURN range(1, maxDegree+1) AS colors
                         """)
    return records[0]['colors']

In [282]:
def get_maximal_fan(x, f1):
    records, _, _ = driver.execute_query("""
    WITH collect{
        MATCH (x:Team {name:$x})-[b:PLAYS]-(f)
        WHERE b.color IS NOT NULL
        RETURN {name:f.name, xRelColor:b.color, incidentColors:COLLECT{ MATCH (f)-[b1:PLAYS]-() WHERE b1.color IS NOT NULL RETURN b1.color}}} AS candidates
    RETURN
        REDUCE(fan=[{name:$f1, incidentColors: COLLECT{ MATCH (:Team {name:$f1})-[b2:PLAYS]-() WHERE b2.color IS NOT NULL RETURN b2.color}}], i in range(0, size(candidates)-1)| 
            fan + coalesce([
                n in range(0, size(candidates)-1) 
                WHERE NOT candidates[n]['xRelColor'] IN fan[-1]['incidentColors']
                AND NOT candidates[n]['name'] IN [fe in fan | fe.name]
                | candidates[n]
            ][0], [])
        ) AS maxFan, [candidate in candidates | candidate['xRelColor']] AS xIncidentColors
                                         """, 
                                         {"x":x, "f1":f1})
    record = records[0]
    return record['maxFan'],  record['xIncidentColors']

In [283]:
def get_fan_candidates(x, f1):
    records, _, _ = driver.execute_query("""
    MATCH (x:Team {name:$x})-[b:PLAYS]-(f)
    WHERE b.color IS NOT NULL OR f.name = $f1
    RETURN f.name AS name, b.color AS xRelColor, 
    COLLECT{ MATCH (f)-[b1:PLAYS]-() WHERE b1.color IS NOT NULL RETURN b1.color} AS incidentColors
    ORDER BY f.name = $f1 DESC""", {"x":x, "f1":f1})
    return records
    

In [284]:
def get_maximal_fan_python(x, f1):
    records = get_fan_candidates(x, f1)
    x_incident_colors = [rec.get('xRelColor') for rec in records if rec.get('xRelColor')]
    fan = [dict(records[0])]
    records = records[1:]
    while True:
        try:
            # Find the next value that meets the condition (e.g., even numbers)
            next_edge = next(record for record in records if not record['xRelColor'] in fan[-1]['incidentColors'])
            
            # Remove the value from candidate
            records.remove(next_edge)
            
            # Add the value to fan
            fan.append(dict(next_edge))
            
        except StopIteration:
            # Break the loop when no more values meet the condition
            break
    return fan, x_incident_colors

In [285]:
def choose_c_d(max_fan, x_incident_colors, colors):
    l_free_colors = [color for color in colors if color not in max_fan[-1]['incidentColors']]
    x_free_colors = [color for color in colors if color not in x_incident_colors]
    c = random.choice(x_free_colors)
    d = random.choice(l_free_colors)
    inversion_required = not (d in x_free_colors)
    return c, d, inversion_required

In [286]:
def invert_c_d_path(x, c, d):
    _, summary, _ = driver.execute_query("""
        MATCH p=(x:Team {name:$x})-[:PLAYS {color:$d}]-()(()-[:PLAYS {color:$c}]-()-[:PLAYS {color:$d}]-())*()-[:PLAYS {color:$c}]-{0,1}()
        WITH p ORDER BY length(p) DESC
        LIMIT 1
        FOREACH(rel in relationships(p) |
        SET rel.color = CASE WHEN rel.color = $c THEN $d ELSE $c END)
                                         """,
                                         {"x": x, "c": c, "d":d})
    return summary.counters

In [287]:
def get_rotate_fan(fan, d):
    rotate_fan_records, _, _ = driver.execute_query("""
        UNWIND RANGE(0, size($fan)-1) AS idx
        MATCH (g:Team {name:$fan[idx]})
        WHERE NOT EXISTS {(g)-[:PLAYS {color:$d}]-()}
        WITH idx LIMIT 1
        RETURN $fan[..idx + 1] AS rotateFan
                                                    """,
                                                    {"fan":fan,
                                                     "d":d})
    record = rotate_fan_records[0]
    return record['rotateFan']

In [288]:
def rotate_and_assign(x, fan, d):
    _, summary, _ = driver.execute_query("""
        MATCH (g:Team {name:$x})
        UNWIND $fan AS f
        MATCH (g)-[r:PLAYS]-(:Team {name:f})
        WITH collect(r) AS rels
        OPTIONAL CALL (rels) {
        UNWIND range(0, size(rels)-2) AS i
        WITH rels[i] AS rel, rels[i+1]['color'] AS color
        SET rel.color = color} 
        WITH rels[-1] AS w
        LIMIT 1
        SET w.color = $d
    """, {"x":x, "fan":fan, "d":d})
    return summary.counters

In [289]:
def color_relationship(x, f1):
    max_fan, x_incident_colors = get_maximal_fan_python(x, f1)
    c, d, inversion_required = choose_c_d(max_fan, x_incident_colors, colors)
    if inversion_required:
       invert_c_d_path(x, c, d)
       rotate_fan = get_rotate_fan([fe['name'] for fe in max_fan], d)    
    else:
        rotate_fan = [fe['name'] for fe in max_fan]
    rotate_and_assign(x, rotate_fan, d)


In [290]:
def validate_coloring():
    records, _, _ = driver.execute_query("""
        MATCH (g:Team)-[b:PLAYS]-()
        WHERE b.color IS NOT NULL
        WITH g, b.color AS color, count(*) as colorCount
        RETURN max(colorCount) AS maxColorsPerNode""")
    print(f"max colors per node {records[0]['maxColorsPerNode']}")
    assert records[0]['maxColorsPerNode'] in [None, 1], "coloring is invalid"

In [291]:
colors = get_colors()

In [292]:
records, _, _ = driver.execute_query("""MATCH (x)-[r:PLAYS]->(f1) WHERE r.color IS NULL RETURN x.name AS x, f1.name AS f1""")
rels = [(record['x'], record['f1']) for record in records]

In [293]:
for rel in rels:
    color_relationship(*rel)

In [294]:
validate_coloring()

max colors per node 1
